##### Loading Libraries

In [1]:

# ******************************* Imports *************************************
# import external libraries
import pandas as pd
import configparser
from datetime import datetime
from pathlib import Path 
import os
import re

import warnings
warnings.filterwarnings('ignore')

# import internal libraries
from Modules.client import RestClient
from Modules.handlers import *

# ****************************** Credentials Config ***************************

def read_config(filename=r'C:\Users\alimes001\Desktop\config.ini'):
    config = configparser.ConfigParser()
    config.read(filename)
    return config['Credentials']

credentials = read_config()
username = credentials['username']
password = credentials['password']

# ****************************** Setting Paths *********************************

file_folder=Path(os.getcwd()).joinpath('Data')
database_folder=Path(os.getcwd()).joinpath('DataBase')
file_folder.mkdir(parents=True, exist_ok=True)
database_folder.mkdir(parents=True, exist_ok=True)
database_file=database_folder.joinpath('task_dict.json')

# ****************************** Instantiate client object *********************
# Define the client object
client = RestClient(username=username, 
                    password=password, 
                    folder_path=file_folder,
                    database_path=database_file)


In [9]:
companies = {
    '585629514':'Zalando',
    '457876088':'Asos',
    '906698760':'farfetch',
    '547951480':'Zara',
    '834465911':'HM',
    '475731082':'Best_Secret'}
# app_ids=[id for id in companies.values()]

In [2]:
##### Create Google/Amazon post

# for Amazon product download
# https://docs.dataforseo.com/v3/merchant/amazon/products/task_post/?python

# For Amazon reviews download
# https://docs.dataforseo.com/v3/merchant/amazon/reviews/task_post/?python

# for Google product download
# https://docs.dataforseo.com/v3/merchant/google/products/task_post/?python

# reviews  >> value to search is a list with asin (list if is multiple_task is True)
# products >> value to search is a list of keywords (list if is multiple_task is True)

create_post(client=client,
            api_type='app_data',
            merchant='apple',
            task_type="app_reviews", 
            value_to_search='585629514', 
            depth=500, 
            multiple_task=False)

Task accepted
Task id: 01110208-7198-0425-0000-0d6e6c2dd71b.


In [3]:
# check the status of the task and download the data
client.task_monitor(
        api_type='app_data',
        merchant='apple',
        task_type='app_reviews',
        time_sec=60*5)

You have 7 tasks in progress
Please wait...
All tasks are downloaded


In [10]:
# 1. read the folder Task-ID
full_df=pd.DataFrame()
task_id_folder = client.folder_path.joinpath('Task-Get','Fake')
for file in task_id_folder.iterdir():
    
    if 'apple_task' in str(file):
        pattern = r'app_reviews_(.*?)\.json'
        match = re.search(pattern, str(file))
        df=from_json_to_df(path=file,is_json=True)
        df['app_id']=match.group(1)
        full_df=pd.concat([full_df,df],axis=0)

# reorder the columns
columns=full_df.columns.tolist()
columns=[columns[-1]]+columns[:-1]
full_df=full_df[columns]


In [11]:
full_df['company']=full_df['app_id'].map(companies)

In [14]:
full_df.to_excel('./Results/AppStore_v2.xlsx',index=False)

In [6]:
full_df.to_excel('./Results/app_store.xlsx', index=False)